# Assignment: Weeks 11 & 12 Exercises

## DSC 540: Data Preparation

## May 31st, 2024

## Kenn Wade

**SQLite Database Creation and Population:**

**Steps:**

   I utilized SQLite, a lightweight relational database management system, to create and populate the `petsdb` database. The objective was to set up the database with sample data for further analysis.

   - First, I created a function to handle database connections with a retry mechanism to manage potential locking issues.
   - Then, I implemented a function to execute SQL commands with retries.
   - The process involved connecting to the SQLite database and dropping any existing tables to ensure a clean setup.
   - I created the `persons` and `pets` tables within the database.
   - After that, I inserted sample data into both tables.
   - Finally, I committed the changes and closed the database connection, ensuring the database was ready for use.

In [10]:
import sqlite3
import time

def connect_to_db(db_name):
    for _ in range(5):  # Retry up to 5 times
        try:
            conn = sqlite3.connect(db_name)
            return conn
        except sqlite3.OperationalError as e:
            print("Database is locked, retrying...")
            time.sleep(1)
    raise Exception("Failed to connect to the database after multiple attempts.")

def execute_with_retry(cursor, query, params=(), retries=5, many=False):
    for _ in range(retries):
        try:
            if many:
                cursor.executemany(query, params)
            else:
                cursor.execute(query, params)
            return
        except sqlite3.OperationalError as e:
            print(f"Error: {e}, retrying...")
            time.sleep(1)
    raise Exception(f"Failed to execute query: {query} after multiple attempts.")

# Step 1: Connect to the SQLite database
print("Step 1: Connecting to the SQLite database...")
conn = connect_to_db('petsdb')
c = conn.cursor()
print("Connected to the SQLite database.")

# Step 2: Drop existing tables if they exist
print("Step 2: Dropping existing tables if they exist...")
try:
    execute_with_retry(c, 'DROP TABLE IF EXISTS persons')
    execute_with_retry(c, 'DROP TABLE IF EXISTS pets')
    print("Existing tables dropped.")
except sqlite3.OperationalError as e:
    print("Error while dropping tables:", e)

# Step 3: Create the 'persons' table
print("Step 3: Creating the 'persons' table...")
execute_with_retry(c, '''
    CREATE TABLE IF NOT EXISTS persons (
        id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        age INTEGER,
        city TEXT,
        zip_code TEXT
    )
''')
print("'persons' table created successfully.")

# Step 4: Create the 'pets' table
print("Step 4: Creating the 'pets' table...")
execute_with_retry(c, '''
    CREATE TABLE IF NOT EXISTS pets (
        owner_id INTEGER,
        pet_name TEXT,
        pet_type INTEGER,
        treatment_done INTEGER
    )
''')
print("'pets' table created successfully.")

# Step 5: Insert data into the 'persons' table
print("Step 5: Inserting data into the 'persons' table...")
execute_with_retry(c, '''
    INSERT INTO persons (id, first_name, last_name, age, city, zip_code)
    VALUES (?, ?, ?, ?, ?, ?)
''', [
    (1, 'Erica', 'Smith', 22, 'south port', '2345678'),
    (2, 'Jordi', 'Johnson', 73, 'east port', '123456'),
    (3, 'Chasity', 'Brown', 70, 'new port', '76856785'),
    (4, 'Gregg', 'Davis', 31, 'new port', '76856785'),
    (5, 'Tony', 'Lindgren', 7, 'west port', '2345678')
], many=True)
print("Data inserted into the 'persons' table successfully.")

# Step 6: Insert data into the 'pets' table
print("Step 6: Inserting data into the 'pets' table...")
execute_with_retry(c, '''
    INSERT INTO pets (owner_id, pet_name, pet_type, treatment_done)
    VALUES (?, ?, ?, ?)
''', [
    (1, 'mani', 1, 1),
    (2, 'tamari', 2, 0),
    (3, 'raba', 3, 1),
    (4, 'olga', 1, 0),
    (5, 'bella', 2, 1)
], many=True)
print("Data inserted into the 'pets' table successfully.")

# Step 7: Commit the changes and close the connection
print("Step 7: Committing changes to the database...")
conn.commit()
conn.close()
print("Database created and populated successfully.")


Step 1: Connecting to the SQLite database...
Connected to the SQLite database.
Step 2: Dropping existing tables if they exist...
Existing tables dropped.
Step 3: Creating the 'persons' table...
'persons' table created successfully.
Step 4: Creating the 'pets' table...
'pets' table created successfully.
Step 5: Inserting data into the 'persons' table...
Data inserted into the 'persons' table successfully.
Step 6: Inserting data into the 'pets' table...
Data inserted into the 'pets' table successfully.
Step 7: Committing changes to the database...
Database created and populated successfully.


**SQLite Database Queries:**

**Steps:**

   I used SQLite to perform a series of queries on the `petsdb` database. The goal was to extract and analyze data from the `persons` and `pets` tables.

   - First, I connected to the SQLite database and verified the existence of the necessary tables.
   - I wrote a function to list the tables and another to check the status of the database connection.
   - The queries executed included:
     - Counting the number of people in each age group.
     - Identifying the age group with the highest number of people.
     - Determining the number of people without a full name.
     - Counting the number of people who own more than one pet.
     - Counting the number of pets that have received treatment.
     - Counting the number of pets that have received treatment and have a known type.
     - Identifying the number of pets from the city called East Port.
     - Identifying the number of pets from East Port that received treatment.
   - Each query was executed, and the results were printed for analysis.
   - The results provided insights into the demographics of the persons and the treatment status of the pets in the database.


In [13]:
import sqlite3

# Step 1: Connect to the SQLite database
print("Step 1: Connecting to the SQLite database...")
conn = sqlite3.connect('petsdb')
c = conn.cursor()
print("Connected to the SQLite database.")

# Step 2: Write a function to check whether the connection has been successful and list tables
def list_tables(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        return tables
    except sqlite3.Error as e:
        print("Error:", e)
        return []

# List the tables in the database
print("Step 2: Listing tables in the database...")
tables = list_tables(conn)
print("Tables in the database:", tables)

# Check if the necessary tables exist before proceeding
if ('persons',) in tables and ('pets',) in tables:
    print("Both 'persons' and 'pets' tables found in the database. Proceeding with queries...")

    # Step 3: Write a function to check whether the connection has been successful
    def is_opened(conn):
        try:
            conn.execute("SELECT * FROM persons LIMIT 1")
            return True
        except sqlite3.ProgrammingError as e:
            print("Connection closed {}".format(e))
            return False

    # Check if the connection is open
    print("Step 3: Checking if the connection is open...")
    print(is_opened(conn))

    # Step 4: Find out the different age groups in the persons table
    print("Step 4: Finding out the different age groups in the persons table...")
    for ppl, age in c.execute("SELECT count(*), age FROM persons GROUP BY age"):
        print("We have {} people aged {}".format(ppl, age))

    # Step 5: Find out which age group has the maximum number of people
    print("Step 5: Finding out which age group has the maximum number of people...")
    for ppl, age in c.execute("SELECT count(*), age FROM persons GROUP BY age ORDER BY count(*) DESC"):
        print("The highest number of people is {} and came from {} age group".format(ppl, age))
        break

    # Step 6: Find out how many people do not have a full name (the last name is blank/null)
    print("Step 6: Finding out how many people do not have a full name (last name is blank/null)...")
    res = c.execute("SELECT count(*) FROM persons WHERE last_name IS null")
    for row in res:
        print("Number of people without a full name:", row[0])

    # Step 7: Find out how many people have more than one pet
    print("Step 7: Finding out how many people have more than one pet...")
    res = c.execute("""
        SELECT count(*) FROM (
            SELECT count(owner_id) FROM pets GROUP BY owner_id HAVING count(owner_id) > 1
        )
    """)
    for row in res:
        print("{} people have more than one pet".format(row[0]))

    # Step 8: Find out how many pets have received treatment
    print("Step 8: Finding out how many pets have received treatment...")
    res = c.execute("SELECT count(*) FROM pets WHERE treatment_done=1")
    for row in res:
        print("Number of pets that have received treatment:", row[0])

    # Step 9: Find out how many pets have received treatment and the type of pet is known
    print("Step 9: Finding out how many pets have received treatment and the type of pet is known...")
    res = c.execute("SELECT count(*) FROM pets WHERE treatment_done=1 AND pet_type IS NOT null")
    for row in res:
        print("Number of pets that have received treatment and the type of pet is known:", row[0])

    # Step 10: Find out how many pets are from the city called east port
    print("Step 10: Finding out how many pets are from the city called east port...")
    res = c.execute("""
        SELECT count(*) FROM pets
        JOIN persons ON pets.owner_id = persons.id
        WHERE persons.city='east port'
    """)
    for row in res:
        print("Number of pets from the city called east port:", row[0])

    # Step 11: Find out how many pets are from the city called east port and who received treatment
    print("Step 11: Finding out how many pets are from the city called east port and who received treatment...")
    res = c.execute("""
        SELECT count(*) FROM pets
        JOIN persons ON pets.owner_id = persons.id
        WHERE persons.city='east port' AND pets.treatment_done=1
    """)
    for row in res:
        print("Number of pets from east port that received treatment:", row[0])

    # Close the database connection
    print("Step 12: Closing the database connection...")
    conn.close()
    print("Database connection closed.")
else:
    print("Required tables not found in the database. Please check the database file and try again.")


Step 1: Connecting to the SQLite database...
Connected to the SQLite database.
Step 2: Listing tables in the database...
Tables in the database: [('persons',), ('pets',)]
Both 'persons' and 'pets' tables found in the database. Proceeding with queries...
Step 3: Checking if the connection is open...
True
Step 4: Finding out the different age groups in the persons table...
We have 1 people aged 7
We have 1 people aged 22
We have 1 people aged 31
We have 1 people aged 70
We have 1 people aged 73
Step 5: Finding out which age group has the maximum number of people...
The highest number of people is 1 and came from 73 age group
Step 6: Finding out how many people do not have a full name (last name is blank/null)...
Number of people without a full name: 0
Step 7: Finding out how many people have more than one pet...
0 people have more than one pet
Step 8: Finding out how many pets have received treatment...
Number of pets that have received treatment: 3
Step 9: Finding out how many pets have

### Summary of Results:

1. **Different age groups in the persons table:**
   - We have 1 person aged 7
   - We have 1 person aged 22
   - We have 1 person aged 31
   - We have 1 person aged 70
   - We have 1 person aged 73

2. **Age group with the maximum number of people:**
   - The highest number of people is 1 and came from the 73 age group.

3. **People without a full name (last name is blank/null):**
   - Number of people without a full name: 0

4. **People with more than one pet:**
   - 0 people have more than one pet

5. **Pets that have received treatment:**
   - Number of pets that have received treatment: 3

6. **Pets that have received treatment and the type of pet is known:**
   - Number of pets that have received treatment and the type of pet is known: 3

7. **Pets from the city called east port:**
   - Number of pets from the city called east port: 1

8. **Pets from the city called east port that received treatment:**
   - Number of pets from east port that received treatment: 0